# The Scoring Logic

# Setup

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import sys
import time
import json
from datetime import datetime
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")
from sentence_transformers import SentenceTransformer, util

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from utils import get_table_from_supabase, build_relationship_cols, build_financial_history
from logic_utils import extract_classifications, get_name_from_id, get_id_from_name, get_granularity_weight, check_if_parent, calculate_similarity_score

#get keys from env
load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

----

# Retrieving Data from Supabase and Building Dataframes

As with my EDA, I will connect to Supabase and retrieve all records, I will create one dataframe for funder information, and another for grants and recipients information. This will allow me to easily access funders' giving history, plus the classifications for both funders and recipients, to be used as part of the calculation of the alignment score.

In [7]:
#get tables and build dataframes
tables = ["funders", "causes", "areas", "beneficiaries", "grants",
               "funder_causes", "funder_areas", "funder_beneficiaries", "funder_grants", 
               "financials", "funder_financials",
               "embedding_pairs", "evaluation_pairs", "logic_pairs",
               "area_hierarchy"]

for table in tables:
    globals()[table] = get_table_from_supabase(url, key, table)

#get recipients with filter
recipients = get_table_from_supabase(url, key, "recipients", batch_size=50, filter_recipients=True)
all_recipient_ids = set(recipients["recipient_id"].unique())

#get and filter recipient join tables
recipient_join_tables = ["recipient_grants", "recipient_areas", "recipient_beneficiaries", "recipient_causes"]
for table in recipient_join_tables:
    df = get_table_from_supabase(url, key, table)
    globals()[table] = df[df["recipient_id"].isin(all_recipient_ids)]

## The Funders Dataframe

### Main Table

In [8]:
funders_df = funders.copy()

#define table relationships for funders
funder_rels = [
    {
        "join_table": funder_causes,
        "lookup_table": causes,
        "key": "cause_id",
        "value_col": "cause_name",
        "result_col": "causes"
    },
    {
        "join_table": funder_areas,
        "lookup_table": areas,
        "key": "area_id",
        "value_col": "area_name",
        "result_col": "areas"
    },
    {
        "join_table": funder_beneficiaries,
        "lookup_table": beneficiaries,
        "key": "ben_id",
        "value_col": "ben_name",
        "result_col": "beneficiaries"
    }
]

#add relationship columns
funders_df = build_relationship_cols(funders_df, "registered_num", funder_rels)

#round to 2 decimal places
funders_df = funders_df.round(2)
pd.set_option("display.float_format", "{:.2f}".format)

### Financial History Table

In [9]:
funders_df = build_financial_history(funders_df, "registered_num", funder_financials, financials)

### The List Entries

In [10]:
#get list entries
list_entries = get_table_from_supabase(url, key, "list_entries")
funder_list = get_table_from_supabase(url, key, "funder_list")
list_with_info = funder_list.merge(list_entries, on="list_id")

#get list of entries for each funder
list_grouped = list_with_info.groupby("registered_num")["list_type"].apply(list).reset_index()
list_grouped.columns = ["registered_num", "list_entries"]

#merge with funders and replace nans
funders_df = funders_df.merge(list_grouped, on="registered_num", how="left")
funders_df["list_entries"] = funders_df["list_entries"].apply(lambda x: x if isinstance(x, list) else [])

In [12]:
#get checkpoint folder
checkpoint_folder = Path("./10.1_checkpoints/")

#create checkpoint - save df to pickle
# funders_df.to_pickle(checkpoint_folder / "funders_df.pkl")
# print("Saved funders_df to checkpoint")

Saved funders_df to checkpoint


## The Grants Dataframe

### Main Table

In [13]:
grants_df = grants.copy()

#ddd funder info
grants_df = grants_df.merge(funder_grants, on="grant_id")
grants_df = grants_df.merge(funders[["registered_num", "name"]], on="registered_num")
grants_df = grants_df.rename(columns={"name": "funder_name", "registered_num": "funder_num"})

#ddd recipient info  
grants_df = grants_df.merge(recipient_grants, on="grant_id")
grants_df = grants_df.merge(recipients[["recipient_id", "recipient_name", "recipient_activities", "recipient_objectives", 
                                        "recipient_name_em", "recipient_activities_em", "recipient_objectives_em", "recipient_concat_em", "is_recipient", "recipient_extracted_class"]], 
                        on="recipient_id", 
                        how="left")

#define relationships for recipients
recipient_rels = [
    {
        "join_table": recipient_areas,
        "lookup_table": areas,
        "key": "area_id",
        "value_col": "area_name",
        "result_col": "recipient_areas"
    },
    {
        "join_table": recipient_causes,
        "lookup_table": causes,
        "key": "cause_id",
        "value_col": "cause_name",
        "result_col": "recipient_causes"
    },
    {
        "join_table": recipient_beneficiaries,
        "lookup_table": beneficiaries,
        "key": "ben_id",
        "value_col": "ben_name",
        "result_col": "recipient_beneficiaries"
    }
]

#add relationship columns
grants_df = build_relationship_cols(grants_df, "recipient_id", recipient_rels)

#add source of grant
grants_df["source"] = grants_df["grant_id"].apply(lambda x: "Accounts" if str(x).startswith("2") else "360Giving")

#round to 2 decimal places
grants_df = grants_df.round(2)

In [15]:
#create checkpoint - save df to pickle
# grants_df.to_pickle(checkpoint_folder / "grants_df.pkl")
# print("Saved grants_df to checkpoint")

Saved grants_df to checkpoint


## The Pairs Dataframe

In [16]:
pairs_df = logic_pairs.copy()

In [17]:
#merge to enrich with funder data
pairs_enriched = pairs_df.merge(
    funders_df,
    left_on="funder_registered_num",
    right_on="registered_num",
    how="left",
    suffixes=("", "_funder")
)

#drop duplicate col
pairs_enriched = pairs_enriched.drop("registered_num", axis=1)

#merge to enrich with recipient data
pairs_enriched = pairs_enriched.merge(
    grants_df[["recipient_id", "recipient_name", "recipient_activities", "recipient_objectives",
                "recipient_areas", "recipient_causes",
                "recipient_beneficiaries", "recipient_extracted_class"]].drop_duplicates(subset=["recipient_id"]),
    on="recipient_id",
    how="left"
)

pairs_df = pairs_enriched.copy()

In [19]:
#create checkpoint - save df to pickle
# pairs_df.to_pickle(checkpoint_folder / "pairs_df.pkl")
# print("Saved pairs_df to checkpoint")

Saved pairs_df to checkpoint


## The Areas Dataframes

In [4]:
#get checkpoint folder
checkpoint_folder = Path("./10.1_checkpoints/")

# areas_df = areas.copy()
# hierarchies_df = area_hierarchy.copy()

#create checkpoint - save dfs to pickle
# areas_df.to_pickle(checkpoint_folder / "areas_df.pkl")
# hierarchies_df.to_pickle(checkpoint_folder / "hierarchies_df.pkl")
# print("Saved areas_df and hierarchies_df to checkpoint")

Saved areas_df and hierarchies_df to checkpoint


---

# Retrieving Data from Checkpoints

In [27]:
#get checkpoint folder
checkpoint_folder = Path("./10.1_checkpoints/")

#get checkpoint
funders_df = pd.read_pickle(checkpoint_folder / "funders_df.pkl")
grants_df = pd.read_pickle(checkpoint_folder / "grants_df.pkl")
pairs_df = pd.read_pickle(checkpoint_folder / "pairs_df.pkl")
areas_df = pd.read_pickle(checkpoint_folder / "areas_df.pkl")
hierarchies_df = pd.read_pickle(checkpoint_folder / "hierarchies_df.pkl")

----

# Preparation of User and Funder Data

## Users

I will use the data for the recipient at index 0 in `pairs_df` as a proxy for a real user's input, to simulate the functionality of the final artefact as I build the logic. 

First, the user will input information about their charity (the applicant), then embeddings will be created for the inputted text data. For the purposes of this development notebook, I will simulate the user's keyword input by using extracted classifications from recipients' data, but in the final artefact, the user will be asked to enter their own keywords.

In [310]:
#simulate user's input
user_df = pairs_df.iloc[[9]][["recipient_id", "recipient_name", "recipient_activities", "recipient_objectives", "recipient_areas", "recipient_causes", "recipient_beneficiaries", "recipient_extracted_class"]]
user_df["funder_num"] = "1075920"

In [311]:
#save registered numbers
user_num = user_df["recipient_id"].iloc[0]
funder_num = user_df["funder_num"].iloc[0]

### Creation of Embeddings from User Input

In [312]:
model = SentenceTransformer("all-roberta-large-v1")
user_cols = ["recipient_name", "recipient_activities", "recipient_objectives"]

for col in user_cols:
    #replace nans with empty string
    texts = user_df[col].fillna("").tolist()
    embeddings = model.encode(texts)
    
    #add to df
    user_df[f"{col}_em"] = list(embeddings)

user_df["concat_text"] = user_df[user_cols[0]].fillna("")
for col in user_cols[1:]:
    user_df["concat_text"] += " " + user_df[col].fillna("")

#make lowercase
user_df["concat_text"] = user_df["concat_text"].str.lower()

#create embeddings
texts = user_df["concat_text"].tolist()
embeddings = model.encode(texts)
user_df["concat_em"] = list(embeddings)

#drop concatenated text
user_df = user_df.drop(columns=["concat_text"])

#change recipient_ to user_
user_df = user_df.rename(columns=lambda col: f"user_{col[len('recipient_'):]}" if col.startswith("recipient_") else col)

In [313]:
pd.set_option("display.max_columns", None)
user_df.head()

,user_id,user_name,user_activities,user_objectives,user_areas,user_causes,user_beneficiaries,user_extracted_class,funder_num,user_name_em,user_activities_em,user_objectives_em,concat_em
9,1102449,WELSH REFUGEE COUNCIL,WE PROMOTE THE INTERESTS OF ASYLUM SEEKERS AND...,A) PROVIDE RELIEF FOR REFUGEES AND THEIR DEPEN...,[Throughout Wales],"[General Charitable Purposes, Education/traini...","[Children/young People, Elderly/old People, Pe...","[""WALES"",""ASYLUM SEEKERS AND REFUGEES"",""CHILDR...",1075920,"[-0.0020253204, 0.0118104, -0.029621243, -0.03...","[0.00028931987, 0.020518208, -0.045814313, -0....","[0.032779414, 0.00608161, -0.044036448, -0.028...","[0.0081218975, 0.007177745, -0.001784812, -0.0..."


## Funders

I will next build a dataframe for the funder selected by the user, and a separate dataframe to store details of previous grants given by, and recipients of, this funder.

In [314]:
#get funder data from number inputted by user
funder_df = funders_df[funders_df["registered_num"] == funder_num].copy()
funder_df.head()

,registered_num,name,website,activities,objectives,income_latest,expenditure_latest,objectives_activities,achievements_performance,grant_policy,is_potential_sbf,is_on_list,is_nua,name_em,activities_em,objectives_em,objectives_activities_em,achievements_performance_em,grant_policy_em,concat_em,extracted_class,causes,areas,beneficiaries,income_history,expenditure_history,list_entries
109,1075920,INDIGO TRUST,https://www.indigotrust.org.uk,GRANT MAKING,TO OR FOR THE BENEFIT OF SUCH ONE OR MORE CHAR...,6682870.0,3721875.0,None,None,None,False,False,False,"[0.0054506254,0.055493865,0.0014405674,0.01435...","[0.014918945,0.083354145,-0.024165146,-0.03651...","[-0.020621957,-0.0005683966,0.007324835,-0.020...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.019817753,-0.00571729,0.022262126,-0.03666...","[-0.023188809,-0.01820599,0.0024579682,-0.0262...","[""GRANT MAKING"",""GRANTS TO ORGANISATIONS""]","[General Charitable Purposes, Other Charitable...","[South Africa, Africa, Scotland, Europe, Throu...",[Other Charities Or Voluntary Bodies],"{2020: 4399659.0, 2021: 2641995.0, 2022: 28440...","{2020: 4183314.0, 2021: 2405175.0, 2022: 17208...",[]


In [315]:
#get grants for selected funder
funder_grants_df = grants_df[grants_df["funder_num"] == funder_num].copy()
funder_grants_df.head(1)

,grant_title,grant_desc,amount,year,grant_id,source,grant_title_em,grant_desc_em,grant_concat_em,grant_extracted_class,funder_num,funder_grants_id,funder_name,recipient_id,recipient_grants_id,recipient_name,recipient_activities,recipient_objectives,recipient_name_em,recipient_activities_em,recipient_objectives_em,recipient_concat_em,is_recipient,recipient_extracted_class,recipient_areas,recipient_causes,recipient_beneficiaries
25357,WRITE TO LIFE PROJECT,TOWARDS THE COSTS OF THE WRITE TO LIFE PROJECT.,23000.0,2002,360G-indigotrust-IND10,360Giving,"[0.027102387,0.0014822506,-0.0038324152,-0.047...","[0.031030053,0.03577807,0.022209514,-0.0081773...","[0.005012915,0.015613875,0.0151255205,-0.01118...",[],1075920,39391,INDIGO TRUST,1000340,38201,FREEDOM FROM TORTURE,FREEDOM FROM TORTURE IS A HUMAN RIGHTS ORGANIS...,3.1 THE OBJECTS OF THE CHARITY ARE FOR THE PUB...,"[0.0014025384,0.060177933,-0.008869013,0.01545...","[0.0012441295,0.03081011,-0.012429256,-0.01267...","[-0.0042924527,0.05366534,-0.030244896,-0.0456...","[-0.018466309,0.035319023,-0.006930157,-0.0269...",True,"[""UK"",""ASYLUM SEEKERS AND REFUGEES"",""FAMILIES""...","[Scotland, Europe, Manchester City, Greater Ma...","[The Advancement Of Health Or Saving Of Lives,...",[Other Defined Groups]


In [316]:
#make dictionary of grants
funder_grants_list = funder_grants_df[[
    "grant_title", "grant_desc", "amount", "year",
    "grant_title_em", "grant_desc_em", "grant_extracted_class",
    "recipient_id", "recipient_name", "recipient_activities", "recipient_objectives",
    "recipient_name_em", "recipient_concat_em",
    "recipient_areas", "recipient_beneficiaries", "recipient_causes", "recipient_extracted_class"
]].to_dict("records")

----

# Binary Criteria

## Step 1: Single-Beneficiary Funders

In [317]:
#check if funder has a single beneficiary
is_sbf = funder_df["is_potential_sbf"].iloc[0]
print(f"Funder has a single beneficiary: {is_sbf}")

Funder has a single beneficiary: False


## Step 2: No Unsolicited Applications

In [318]:
#check if funder states no unsolicited applications
is_nua = funder_df["is_nua"].iloc[0]
print(f"Funder states no unsolicited applications: {is_nua}")
print(f"NUA score: {is_nua * 1.0}")

Funder states no unsolicited applications: False
NUA score: 0.0


## Step 3: The List

In [319]:
#check if funder is on the list
def check_is_on_list(funder_df):

    is_on_list = funder_df["is_on_list"].iloc[0]
    reasoning = []

    if is_on_list:
        reasoning = set(funder_df["list_entries"].iloc[0])
    else:
        reasoning = None
    
    score = 0.0 if is_on_list else 1.0
    
    return is_on_list, reasoning, score

is_on_list, list_reasoning, list_score = check_is_on_list(funder_df)
print(f"Funder is on The List: {is_on_list}")
print(f"Type of List entry: {list_reasoning}")
print(f"Score: {list_score}")

Funder is on The List: False
Type of List entry: None
Score: 1.0


## Step 4: Existing Relationship

In [320]:
#check if funder has ever given a grant to applicant
existing_relationship = False
relationship = grants_df[
    (grants_df["funder_num"] == funder_num) &
    (grants_df["recipient_id"] == user_num)
]
num_grants = len(relationship)

if num_grants > 0:
    existing_relationship = True

print(f"Funder and user have existing relationship: {existing_relationship}")
print(f"Funder has given {num_grants} grant(s) to user")

Funder and user have existing relationship: False
Funder has given 0 grant(s) to user


----

# Classification Criteria

## Step 5: Areas

Scoring for areas uses hierarchical matching to account for parent-child geographic relationships. The granularity of each area affects its weight - specific locations like local authorities score higher (1.0) than broad regions (0.7). 

I will check three types of matches:
- Exact matches where both funder and user state that they work in the same area
- Hierarchical matches where the funder's area contains the user's (e.g., funder says "Throughout England", user works in "Bristol")
- Hierarchical matches where the user's area contains the funder's specific location (e.g., user works throughout "Africa", funder focuses on "Kenya"). 

Each match will be weighted differently to reflect the strength of the match. The final score will average only the matched areas, ignoring non-matches, so that having some high-quality geographic alignment is valued over penalising for coverage gaps.

In [321]:
def check_areas(funder_list, user_list, areas_df, hierarchies_df):
    """
    Calculates a score based on matches between the funder's and user's stated areas.
    """

    #convert names to ids
    funder_ids = [get_id_from_name(name, areas_df) for name in funder_list if get_id_from_name(name, areas_df) is not None]
    user_ids = [get_id_from_name(name, areas_df) for name in user_list if get_id_from_name(name, areas_df) is not None]
    
    #avoid zero division
    if len(user_ids) == 0:
        return 0.0, []
    
    #store ids as set and scores/reasoning as lists
    funder_set = set(funder_ids)
    scores = []
    reasoning = []
    
    for user_area in user_ids:
        user_area_name = get_name_from_id(user_area, areas_df)
        
        #check for exact match
        if user_area in funder_set:
            score = get_granularity_weight(user_area, areas_df) * 1.0
            scores.append(score)
            reasoning.append(f"Exact match: {user_area_name}")
        
        #check if user area is within funder area
        else:
            hierarchy_user_in_funder = None
            for funder_area in funder_ids:
                if check_if_parent(funder_area, user_area, hierarchies_df):
                    hierarchy_user_in_funder = funder_area
                    break
            
            if hierarchy_user_in_funder:
                parent_name = get_name_from_id(hierarchy_user_in_funder, areas_df)
                score = get_granularity_weight(hierarchy_user_in_funder, areas_df) * 0.6
                scores.append(score)
                reasoning.append(f"Hierarchical match: {user_area_name} (user) within {parent_name} (funder)")
            
            #check if funder area is within user area
            else:
                hierarchy_funder_in_user = None
                for funder_area in funder_ids:
                    if check_if_parent(user_area, funder_area, hierarchies_df):
                        hierarchy_funder_in_user = funder_area
                        break
                
                if hierarchy_funder_in_user:
                    child_name = get_name_from_id(hierarchy_funder_in_user, areas_df)
                    score = get_granularity_weight(user_area, areas_df) * 0.4
                    scores.append(score)
                    reasoning.append(f"Hierarchical match: {child_name} (funder) within {user_area_name} (user)")
                
                #no match
                else:
                    scores.append(0.0)
                    reasoning.append(f"No match: {user_area_name}")
    
    matched_scores = [s for s in scores if s > 0]
    if len(matched_scores) > 0:
        score = sum(matched_scores) / len(matched_scores)
    else:
        score = 0.0
    
    return score, reasoning

#get lists
funder_areas = funder_df["areas"].iloc[0].copy()
user_areas = user_df["user_areas"].iloc[0].copy()

#get score and reasoning
areas_score, areas_reasoning = check_areas(funder_areas, user_areas, areas_df, hierarchies_df)
print(f"Areas score: {areas_score:.2f}\n\nReasoning:")
for reason in areas_reasoning:
    print(reason)

Areas score: 0.00

Reasoning:
No match: Throughout Wales


## Step 6: Beneficiaries

Scoring for beneficiaries is simpler than for areas. I will exclude the generic "Other Charities Or Voluntary Bodies" as it is likely that almost all funders will fall into this category, adding noise to the scoring. I will use a hierarchical scoring approach but without the granularity weighting, as the higher level categories in this classification are too broad as to offer real value to the calculation. 

In [322]:
def check_beneficiaries(funder_list, user_list):
    """
    Calculates a score based on matches between the funder's and user's stated beneficiaries.
    """

    #define categories and filter
    high_level_bens = {"Other Defined Groups", "The General Public/mankind"}
    exclude_bens = {"Other Charities Or Voluntary Bodies"}
    funder_bens = [ben for ben in funder_list if ben not in exclude_bens]
    user_bens = [ben for ben in user_list if ben not in exclude_bens]
    
    #avoid zero division
    if len(user_bens) == 0:
        return 0.0, []
    
    #categorise funder beneficiaries
    funder_specific = set(ben for ben in funder_bens if ben not in high_level_bens)
    has_high_level = any(ben in high_level_bens for ben in funder_bens)
    
    scores = []
    reasoning = []
    for user_ben in user_bens:
        if user_ben in funder_specific:
            scores.append(1.0)
            reasoning.append(f"Exact match: {user_ben}")
        elif has_high_level:
            scores.append(0.2)
            reasoning.append(f"Weak match: user states '{user_ben}' and funder supports broad categories")
        else:
            scores.append(0.0)
            reasoning.append(f"No match: {user_ben}")
    
    matched_scores = [s for s in scores if s > 0]
    if len(matched_scores) > 0:
        score = sum(matched_scores) / len(matched_scores)
    else:
        score = 0.0

    return score, reasoning

#get lists
funder_beneficiaries = funder_df["beneficiaries"].iloc[0].copy()
user_beneficiaries = user_df["user_beneficiaries"].iloc[0].copy()

#get score and reasoning
beneficiaries_score, beneficiaries_reasoning = check_beneficiaries(funder_beneficiaries, user_beneficiaries)
print(f"Beneficiaries score: {beneficiaries_score:.2f}\n\nReasoning:")
for reason in beneficiaries_reasoning:
    print(reason)

Beneficiaries score: 0.00

Reasoning:
No match: Children/young People
No match: Elderly/old People
No match: People Of A Particular Ethnic Or Racial Origin


## Step 7: Causes

For causes, I will exclude "Other Charitable Purposes" as it adds noise. However, I will not exclude "General Charitable Purposes" (GCP) as this is used by funders to indicate that they would be willing to consider any causes. I will use it as a fallback similar to how "Throughout England" works for areas. 

The scoring checks for exact matches between the user's and funder's causes first, which score 1.0. If no exact match exists but the funder lists GCP, this scores 0.5 as a weak indicator that the funder might support the cause. Non-matches score 0.0.

In [323]:
def check_causes(funder_list, user_list):
    """
    Calculates a score based on matches between the funder's and user's stated causes.
    """
    #define categories and filter
    gcp = "General Charitable Purposes"
    exclude_causes = {"Other Charitable Purposes"}
    funder_causes = [cause for cause in funder_list if cause not in exclude_causes]
    user_causes = [cause for cause in user_list if cause not in exclude_causes]
    
    #avoid zero division
    if len(user_causes) == 0:
        return 0.0, []
    
    #categorise funder causes
    funder_specific = set(cause for cause in funder_causes if cause != gcp)
    has_gcp = gcp in funder_causes
    
    scores = []
    reasoning = []
    
    for user_cause in user_causes:
        if user_cause in funder_specific:
            scores.append(1.0)
            reasoning.append(f"Exact match: {user_cause}")
        elif has_gcp:
            scores.append(0.6)
            reasoning.append(f"Weak match: user states '{user_cause}' and funder supports general charitable purposes")
        else:
            scores.append(0.0)
            reasoning.append(f"No match: {user_cause}")
    
    matched_scores = [s for s in scores if s > 0]
    if len(matched_scores) > 0:
        score = sum(matched_scores) / len(matched_scores)
    else:
        score = 0.0
    
    return score, reasoning

#get lists
funder_causes = funder_df["causes"].iloc[0].copy()
user_causes = user_df["user_causes"].iloc[0].copy()

#get score and reasoning
causes_score, causes_reasoning = check_causes(funder_causes, user_causes)
print(f"Causes score: {causes_score:.2f}\n\nReasoning:")
for reason in causes_reasoning:
    print(reason)

Causes score: 0.60

Reasoning:
Weak match: user states 'General Charitable Purposes' and funder supports general charitable purposes
Weak match: user states 'Education/training' and funder supports general charitable purposes
Weak match: user states 'The Prevention Or Relief Of Poverty' and funder supports general charitable purposes
Weak match: user states 'Arts/culture/heritage/science' and funder supports general charitable purposes
Weak match: user states 'Economic/community Development/employment' and funder supports general charitable purposes


-----

# Semantic Similarity

## Step 8: Text Sections

In [324]:
#get embeddings
funder_embedding = funder_df["concat_em"].iloc[0]
user_embedding = user_df["concat_em"].iloc[0]

#get score
text_similarity_score = calculate_similarity_score(funder_embedding, user_embedding)
print(f"Text semantic similarity score: {text_similarity_score:.2f}")

Text semantic similarity score: 0.21


## Step 9: Keywords

In [325]:
def check_keywords(funder_keywords, user_keywords, model):
    """
    Calculates semantic similarity between funder (extracted) and user (inputted) keywords.
    """
    
    #parse json
    if isinstance(funder_keywords, str):
        funder_keywords = json.loads(funder_keywords)
    if isinstance(user_keywords, str):
        user_keywords = json.loads(user_keywords)
    
    #handle empty/nans
    if not funder_keywords:
        funder_keywords = []
    if not user_keywords:
        user_keywords = []
    
    if len(funder_keywords) == 0 or len(user_keywords) == 0:
        return 0.0, {}, ["No keywords to compare"]
    
    #create embeddings for each keyword
    funder_keywords_em = {}
    for keyword in funder_keywords:
        embedding = model.encode(keyword)
        funder_keywords_em[keyword] = embedding

    user_keywords_em = {}
    for keyword in user_keywords:
        embedding = model.encode(keyword)
        user_keywords_em[keyword] = embedding

    #compare every funder keyword to every user keyword
    all_scores = []
    for funder_kw, funder_em in funder_keywords_em.items():
        for user_kw, user_em in user_keywords_em.items():
            similarity = calculate_similarity_score(funder_em, user_em)
            all_scores.append({
                "funder_keyword": funder_kw,
                "user_keyword": user_kw,
                "similarity": similarity
            })
    
    #sort and check for bonus (matches >= 0.8)
    all_scores.sort(key=lambda x: x["similarity"], reverse=True)
    gets_bonus = any(match["similarity"] >= 0.80 for match in all_scores)
    
    #get dictionary of matches >= 0.80
    strong_matches = {}
    for match in all_scores:
        if match["similarity"] >= 0.80:
            key = f"{match['funder_keyword']} & {match['user_keyword']}"
            strong_matches[key] = match["similarity"]
    
    #filter to top 10 matches <= 0.80 and get average
    scores_under_80 = [match for match in all_scores if match["similarity"] < 0.80]
    top_10 = scores_under_80[:10]

    if len(top_10) > 0:
        score = sum(match["similarity"] for match in top_10) / len(top_10)
    else:
        score = 0.0
    
    #build reasoning from medium matches
    reasoning = []
    for match in scores_under_80[:9]:
        reasoning.append(f"'{match['funder_keyword']}' & '{match['user_keyword']}': {match['similarity']:.3f}")
    
    return score, strong_matches, reasoning, gets_bonus

#get keyword lists
funder_keywords = funder_df["extracted_class"].iloc[0]
user_keywords = user_df["user_extracted_class"].iloc[0]

#get score
keyword_similarity_score, keyword_strong_matches, keyword_reasoning, keyword_gets_bonus = check_keywords(funder_keywords, user_keywords, model)

print(f"Keyword similarity score: {keyword_similarity_score:.2f}")
print(f"Eligible for keyword bonus: {keyword_gets_bonus}")


Keyword similarity score: 0.31
Eligible for keyword bonus: False


-----

# Revealed Preferences

In [327]:


def check_recipients_rp():
    pass

def check_keywords_rp():
    pass

def calculate_missing_penalty_rp():
    pass

def calculate_keywords_bonus_rp():
    pass

### Name

In [336]:
#get embeddings
recipients_all_em = dict(zip(funder_grants_df["recipient_name"], funder_grants_df["recipient_name_em"]))
user_name_em = user_df["user_name_em"].iloc[0]
user_name = user_df["user_name"].iloc[0]

#get score
name_score_rp, name_reasoning_rp = check_name_rp(recipients_all_em, user_name_em, user_name)
print(f"Name (RP) similarity score: {name_score_rp:.2f}")

Name (RP) similarity score: 0.61


### Grants

In [343]:
#get embeddings
grants_all_em = dict(zip(funder_grants_df["recipient_name"], funder_grants_df["grant_concat_em"]))
user_concat_em = user_df["concat_em"].iloc[0]
user_name = user_df["user_name"].iloc[0]

#get score
grants_rp_score, grants_rp_reasoning = check_grants_rp(grants_all_em, user_concat_em, user_name)
print(f"Grants (RP) similarity score: {grants_rp_score:.2f}")

Grants (RP) similarity score: 0.49


In [329]:
#full function
def calculate_revealed_preferences():
    pass

#check for grants history
if funder_grants_df.empty:
    revealed_preferences_score = 0.0
else:
    revealed_preferences_score = calculate_revealed_preferences()

-----

# Penalties and Bonuses

## SBF Penalty

In [330]:
sbf_penalty = 0.2 if is_sbf else 1.0
print(f"Single-beneficiary penalty: *{sbf_penalty}")

Single-beneficiary penalty: *1.0


## Low Variance Penalty

In [331]:
def calculate_lv_penalty(funder_grants_df):
    """
    Identifies low variance in a funder's previous giving and calculates a penalty.
    """
    pass

## Keywords Bonus

In [332]:
def calculate_keywords_bonus(strong_matches):
    """
    Calculates bonus based on keyword matches. Only runs if keywords with semantic scores above 0.8 exist.
    """
    pass

## Relationship Bonus

In [333]:
def calculate_relationship_bonus(relationship_df):
    """
    Calculates time since last grant and calculates a bonus. Only runs if there is a relationship.
    """

    #get time lapsed since last gift
    last_grant_year = relationship_df["year"].max()
    current_year = datetime.now().year
    time_lapsed = current_year - last_grant_year
    
    #normalise
    if time_lapsed <= 2:
        bonus = 1.5
    elif time_lapsed <= 3:
        bonus = 1.4
    elif time_lapsed <= 5:
        bonus = 1.3
    elif time_lapsed <= 10:
        bonus = 1.2
    else:
        bonus = 1.1
    
    #add uplift for recurring relationship
    num_grants = len(relationship_df)
    if num_grants >= 5:
        bonus += 0.1
    
    return time_lapsed, bonus, last_grant_year

#get bonus and reasoning
time_lapsed, relationship_bonus, last_grant_year = calculate_relationship_bonus(relationship)

print(f"Last gift given in {last_grant_year} ({time_lapsed} years ago)")
print(f"Relationship bonus: *{relationship_bonus}")

Last gift given in nan (nan years ago)
Relationship bonus: *1.1
